### 라이브러리 임포트

In [1]:
import pandas as pd
import requests
from io import StringIO
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import random

### 파일을 불러오는 방법 4가지
* CSV 파일을 로컬에서 업로드하는 경우
* CSV 파일을 웹주소를 이용해 불러오는 경우
* XLS 파일을 로컬에서 업로드하는 경우
* CSV 파일을 웹주소를 이용해 불러오는 경우

In [2]:
## Code
#### CSV 파일 로컬 업로드 ####
csv_local = "C:\\AI_Dataset\\mobile.csv"
csv_local_df = pd.read_csv(csv_local)
print(f"##### csv_local_df #####")
print(csv_local_df.head())
print()

#### CSV 파일 웹에서 로드 ####
csv_web = "https://github.com/MyungKyuYi/AI-class/raw/main/mobile.csv"
csv_web_df = pd.read_csv(csv_web)
print(f"##### csv_web_df #####")
print(csv_web_df.head())
print()

#### XLS 파일 로컬 업로드 ####
xls_local = "C:\\AI_Dataset\\combined_dataset-1.xlsx"
xls_local_df = pd.read_excel(xls_local) # openpyxl 라이브러리 필요
print(f"##### xls_local_df #####")
print(xls_local_df.head())
print()

#### XLS 파일 웹에서 로드 ####
xls_web = "https://github.com/MyungKyuYi/AI-class/raw/main/combined_dataset-1.xlsx"
xls_web_df = pd.read_excel(xls_web) # openpyxl 라이브러리 필요
print(f"##### xls_web_df #####")
print(xls_web_df.head())

##### csv_local_df #####
   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2  ...         20       756  2549     9     7         19   
1        136        3  ...        905      1988  2631    17     3          7   
2        145        5  ...       1263      1716  2603    11     2          9   
3        131        6  ...       1216      1786  2769    16     8         11   
4        141        2  ...       1208      1212  1411     8     2         15   

   three_g  t

### 타이타닉 데이터 분류
목표:
* 결측치(null값) 확인 후 임의의 값으로 입력
* 레이블 확인 후 imbalanced data 체크
* 타이타닉 데이터에서 필요없는 열(column) 제거
* 숫자로 인코딩
* DT, RF, SVM, LR, KNN 분류 수행
* accuracy, confusion matrix 확인

In [3]:
# 파일을 불러오고 제대로 불러와졌는지 확인
titanic = "C:\\AI_Dataset\\titanic.csv"
titanic_df = pd.read_csv(titanic)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# 결측치 확인
print(titanic_df.isnull().sum())
# Cabin은 추후 삭제할 열이므로 결측치를 제거하지 않음
# Age는 필요한 열이므로 평균치로 채워 결측치를 제거함

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [5]:
# 결측치 제거
# 참고: https://cyber.gachon.ac.kr/mod/ubboard/article.php?id=942049&bwid=848071

mean_age = titanic_df['Age'].mean()

# age 컬럼의 결측치를 평균값으로 채우기

titanic_df['Age'] = titanic_df['Age'].fillna(mean_age)

print(titanic_df.isnull().sum()) # 결측치 다시 체크

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [6]:
# 필요없는 열 제거
drop_columns = ["Name", "Ticket", "Cabin", "Embarked"]
titanic_df = titanic_df.drop(columns = drop_columns)
print(titanic_df.head())

   PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch     Fare
0            1         0       3    male  22.0      1      0   7.2500
1            2         1       1  female  38.0      1      0  71.2833
2            3         1       3  female  26.0      0      0   7.9250
3            4         1       1  female  35.0      1      0  53.1000
4            5         0       3    male  35.0      0      0   8.0500


In [7]:
# imbalanced data 확인
print(titanic_df['Sex'].value_counts())
# 남성이 약 260명가량 더 많음

Sex
male      577
female    314
Name: count, dtype: int64


In [8]:
# 데이터 인코딩(범주형 데이터를 숫자로 변환)
encoder = LabelEncoder()
titanic_df['Sex'] = encoder.fit_transform(titanic_df['Sex'])
print(titanic_df.head())
# Sex 항목이 1과 0으로 변환됨(male = 1, female = 0)

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare
0            1         0       3    1  22.0      1      0   7.2500
1            2         1       1    0  38.0      1      0  71.2833
2            3         1       3    0  26.0      0      0   7.9250
3            4         1       1    0  35.0      1      0  53.1000
4            5         0       3    1  35.0      0      0   8.0500


In [9]:
# X_train, y_train, X_test, y_test 분리
X = titanic_df.drop(columns=['Survived'])
y = titanic_df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# 모델 학습 및 평가

# 모델 리스트
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "Logistic Regression": LogisticRegression(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

# 모델 학습 및 평가
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")
    print(f"{name} Confusion Matrix:\n{cm}\n")

Decision Tree Accuracy: 0.7263
Decision Tree Confusion Matrix:
[[83 22]
 [27 47]]

Random Forest Accuracy: 0.7989
Random Forest Confusion Matrix:
[[92 13]
 [23 51]]

Support Vector Machine Accuracy: 0.5978
Support Vector Machine Confusion Matrix:
[[103   2]
 [ 70   4]]

Logistic Regression Accuracy: 0.7765
Logistic Regression Confusion Matrix:
[[89 16]
 [24 50]]

K-Nearest Neighbors Accuracy: 0.6592
K-Nearest Neighbors Confusion Matrix:
[[89 16]
 [45 29]]



C:\Users\gustj\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 타이타닉 데이터를 이용해서 예측값과 정답 비교(10개)

In [11]:
# 예측값과 정답 비교

# 테스트 데이터에서 10개 샘플 선택
X_sample = X_test[:10]
y_sample_true = y_test.iloc[:10]

# 모델 선택 (예: Random Forest)
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_sample_pred = model.predict(X_sample)

# 결과 비교
comparison = pd.DataFrame({
    "Actual": y_sample_true.values,
    "Predicted": y_sample_pred
})
comparison["Result"] = comparison["Actual"] == comparison["Predicted"]

print(comparison)

   Actual  Predicted  Result
0       1          0   False
1       0          0    True
2       0          0    True
3       1          1    True
4       1          0   False
5       1          1    True
6       1          1    True
7       0          0    True
8       1          1    True
9       1          1    True
